# Merging Data

### Objectives
After this lesson you should be able to...
+ Insert a column into a precise location in a DataFrame
+ Concatenate multiple DataFrames vertically and horizontally
+ Use basic SQL-style joins with **`merge`**
+ Use pandas **`join`** to combine datasets
+ Know the difference between **`join`** and **`merge`**


### Prepare for this lesson by...
+ Read entire page on [Merge, join, and concatenate](http://pandas.pydata.org/pandas-docs/stable/merging.html)

### Introduction
Most all data analysis will deal with multiple different datasets or at least multiple datasets created from the same source. Pandas has tools to merge and combine DataFrames in a wide variety of ways. The only application of combining new data thus far was by adding an additional column. 

### Adding a column - a review
Let's review adding a new column. We will work with the college data set and find the total SAT score.

In [6]:
import pandas as pd
import numpy as np

college = pd.read_csv('data/college.csv')
pd.options.display.max_columns = 40

In [8]:
college.head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,5451.0,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,4811.0,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [19]:
# create new column with SAT total score

college['SAT_TOTAL'] = college.SATMTMID + college.SATVRMID

In [10]:
# check the right side of the data frame
college.iloc[:5, -5:]

,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP,SAT_TOTAL
0,0.8284,0.1049,30300,33888,844.0
1,0.5214,0.2422,39700,21941.5,1135.0
2,0.7795,0.8540,40100,23370,NaN
3,0.4596,0.2640,45500,24097,1185.0
4,0.7554,0.1270,26600,33118.5,855.0


### Inserting a column somewhere else besides the end

The default behavior for adding a new column is to make it the last column. Pandas provides the **`insert`** method to insert a column in a specific place. Pandas can also remove columns with the **`drop`** method. Let's first drop the column we just added and the insert the `SAT_TOTAL` column next to the other SAT columns.

**`drop`** can be used to remove columns or rows (by index) and just like any other method that can operate on columns or indexes, the **`axis`** argument is available. Again, **`axis`** can be set with 0\1 or the strings 'index'\'columns'. The default for **`axis`** is always 0, which in the case of **`drop`** will attempt to find an index with the given label.

In [37]:
# drop a column

# college.drop('SAT_TOTAL') will produce a error. SAT_TOTAL is not in the index

# this will not produce an error if SAT_TOTAL is not found
college.drop('SAT_TOTAL', axis='columns', errors='ignore', inplace=True)

In [38]:
# check that SAT_TOTAL is dropped
'SAT_TOTAL' in college.columns

False

### Insert only takes integer location
The **`insert`** method works by using the integer location of where the new column will be placed. So, knowing the name of the column immediately before the insertion point is not enough information. The **`get_loc`** Series method must be used to retrieve the integer location of the new column. **`insert`** is a rare method that always happens in place.

In [39]:
# we want to insert new column after SATMTMID
new_loc = college.columns.get_loc('SATMTMID') + 1

In [40]:
# now insert in specific location - always happens in place
college.insert(loc=new_loc, column='SAT_TOTAL', value=college.SATMTMID + college.SATVRMID)

In [41]:
# print out result
college.iloc[:5, :10]

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,SAT_TOTAL
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,844.0
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,1135.0
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,NaN
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,1185.0
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,855.0


## Concatenating Data
[Concatenating data](http://pandas.pydata.org/pandas-docs/stable/merging.html) in Pandas refers to stacking DataFrames either one on top of each other or side by side. The **`pd.concat`** function is flexible and versatile with many different arguments that give you power to combine two ore more datasets at the same time.


### Concatenating very similar DataFrames
**`pd.concat`** provides many different and sometimes confusing arguments. First we will grab some dummy stock market data and put the **Symbol** column into the index.

In [83]:
# Get some fake stock data
stocks_2014 = pd.read_csv('data/stocks/stocks_2014.csv', index_col='Symbol')
stocks_2015 = pd.read_csv('data/stocks/stocks_2015.csv', index_col='Symbol')

In [67]:
stocks_2014

,Shares,Low,High
Symbol,,,
AAPL,80,95,110
TSLA,50,80,130
WMT,40,55,70


In [68]:
stocks_2015

,Shares,Low,High
Symbol,,,
AAPL,50,120,140
GE,100,30,40
IBM,87,75,95
SLB,20,55,85
TXN,500,15,23
TSLA,100,100,300


### Default concat
The first argument, **`objs`** for concat must be a sequence, and usually is just a list of DataFrames or Series that are to be combined. The **`axis`** argument controls whether concatenation happens up and down or side to side. It defaults to 0 and stacks the objects vertically.

### Automatica Alignment of Index
Of extreme importance to **`pd.concat`** (and all of pandas) is the automatic alignment of indexes that happens behind the scenes. When concatenating vertically (axis=0), the column names are first aligned and then concatenation happens.

In [104]:
# stack the objects vertically
# give list of dataframes

stocks_1415 = pd.concat([stocks_2014, stocks_2015])

stocks_1415

,Shares,Low,High
Symbol,,,
AAPL,80,95,110
TSLA,50,80,130
WMT,40,55,70
AAPL,50,120,140
GE,100,30,40
IBM,87,75,95
SLB,20,55,85
TXN,500,15,23
TSLA,100,100,300


### Labeling Each Piece
The above output is exactly as expected. A straight stack of one DataFrame on top of another. One obvious problem is that now the index contains multiple instances of some of the symbols. Selecting 'TSLA' produces a 2 row DataFrame. This may or may not be what we want.

In [106]:
# output all the TSLA
stocks_1415.loc['TSLA']

,Shares,Low,High
Symbol,,,
TSLA,50,80,130
TSLA,100,100,300


### Identify the concatenating DataFrames
It's possible to add a level to the index of the resulting DataFrame from concatenation. The **`keys`** argument takes a list of strings that uniquely identify the joining DataFrames. The resulting DataFrame now contains a MultiIndex that identifies how the concatenation happened.

In [135]:
# Add level with keys and name the index with `names` argument

stocks_1415_2 = pd.concat([stocks_2014, stocks_2015], keys=['2014', '2015'], names=['Year'])
stocks_1415_2

Shares  Low  High
Year Symbol                   
2014 AAPL        80   95   110
     TSLA        50   80   130
     WMT         40   55    70
2015 AAPL        50  120   140
     GE         100   30    40
     IBM         87   75    95
     SLB         20   55    85
     TXN        500   15    23
     TSLA       100  100   300

### MultiIndex Selecting
Selecting specific rows [from a MultiIndex](http://pandas.pydata.org/pandas-docs/stable/advanced.html) is unfortunately a complicated affair and another notebook will be dedicated to it. Typically **.loc** is used

In [166]:
# must sort index first for selecting
stocks_1415_2.sort_index(inplace=True)

In [167]:
# Here are a few examples
# get all 2014 data

stocks_1415_2.loc[['2014']]

Shares  Low  High
Year Symbol                   
2014 AAPL        80   95   110
     TSLA        50   80   130
     WMT         40   55    70

In [168]:
# get 2104 AAPL
stocks_1415_2.loc[[('2014', 'AAPL')], :]

,,Shares,Low,High
Year,Symbol,,,
2014,AAPL,80,95,110


In [169]:
# Get all AAPL Data
stocks_1415_2.loc[(slice(None), 'AAPL'), :]

,,Shares,Low,High
Year,Symbol,,,
2014,AAPL,80,95,110
2015,AAPL,50,120,140


### Concatenating horizontally
It's also possible to stack frames horizontally by changing the **`axis`** argument. Automatic alignment of indexes happen first.

In [171]:
# lets see what happens
pd.concat([stocks_2014, stocks_2015], axis=1)

,Shares,Low,High,Shares,Low,High
AAPL,80.0,95.0,110.0,50.0,120.0,140.0
GE,NaN,NaN,NaN,100.0,30.0,40.0
IBM,NaN,NaN,NaN,87.0,75.0,95.0
SLB,NaN,NaN,NaN,20.0,55.0,85.0
TSLA,50.0,80.0,130.0,100.0,100.0,300.0
TXN,NaN,NaN,NaN,500.0,15.0,23.0
WMT,40.0,55.0,70.0,NaN,NaN,NaN


### Missing values when indices don't align.

The two frames only have two values in their indexes that are the same (AAPL and TSLA). The index is the first thing that is created and is the **union** of all the indexes in both frames. Missing values will be placed at every place where either of the frames was missing that index.

In [174]:
# Pandas can do the union for you
stocks_2014.index.union(stocks_2015.index)

Index(['AAPL', 'GE', 'IBM', 'SLB', 'TSLA', 'TXN', 'WMT'], dtype='object', name='Symbol')

In [179]:
### Add keys and level naems again

stocks_1415_wide = pd.concat([stocks_2014, stocks_2015], axis=1, keys=['2014', '2015'], names=['Year'])
stocks_1415_wide

Year   2014                2015              
     Shares   Low   High Shares    Low   High
AAPL   80.0  95.0  110.0   50.0  120.0  140.0
GE      NaN   NaN    NaN  100.0   30.0   40.0
IBM     NaN   NaN    NaN   87.0   75.0   95.0
SLB     NaN   NaN    NaN   20.0   55.0   85.0
TSLA   50.0  80.0  130.0  100.0  100.0  300.0
TXN     NaN   NaN    NaN  500.0   15.0   23.0
WMT    40.0  55.0   70.0    NaN    NaN    NaN

### Changing type of index alignment

By default, **`pd.concat`** will do a union of the indexes. Thinking in SQL this can be thought of as an **outer join**. Concatenation can also happen via intersection of the indexes (only those in common) - an **inner join**.

In [184]:
# do intersection
stocks_2014.index.intersection(stocks_2015.index)

Index(['AAPL', 'TSLA'], dtype='object', name='Symbol')

In [182]:
## concat only common indices
pd.concat([stocks_2014, stocks_2015], axis=1, join='inner', keys=['2014', '2015'], names=['Year'])

Year     2014            2015          
       Shares Low High Shares  Low High
Symbol                                 
AAPL       80  95  110     50  120  140
TSLA       50  80  130    100  100  300

### DataFrame methods vs Pandas Functions

If you noticed, **`pd.concat`** is a function. There is a similar DataFrame **method** called **`append`** . The documentation states that **`append`** predates **`concat`**, can only add rows and does not have as many arguments either so append is really only useful in the most simple cases.

In [220]:
# output the difference in types

type(pd.concat), type(stocks_2014.append)

(function, method)

In [228]:
# append can only do simple row appending. No keys argument to differentiate
stocks_2014.append(stocks_2015)

,Shares,Low,High
Symbol,,,
AAPL,80,95,110
TSLA,50,80,130
WMT,40,55,70
AAPL,50,120,140
GE,100,30,40
IBM,87,75,95
SLB,20,55,85
TXN,500,15,23
TSLA,100,100,300


In [ ]:
# and
stocks_2014.append(stocks_2015)

### SQL-style joins with `merge` and `join`

Many people will come to learn Pandas after learning SQL (structured query language). A very important component of SQL is its ability to join tables in any way imaginable. Pandas provides two similar pieces of functionality with **`merge`** and **`join`**.

**`merge`** is both a pandas function and a DataFrame method that do the exact same thing. It joins tables horizontally by aligning column or index values. **`merge`** cannot be used to stack two frames on top of one another like **`concat`** and **`append`**.

In [259]:
# move the index out as a column to have key to join on
stocks_2014 = pd.read_csv('data/stocks/stocks_2014.csv')
stocks_2015 = pd.read_csv('data/stocks/stocks_2015.csv')

In [262]:
# use the dataframe method
# argument `on` specifies the column to join on
# default is inner join - value must be present in both tables
# x/y are appended to left/right frame to identify columns

stocks_2014.merge(stocks_2015, on='Symbol')

,Symbol,Shares_x,Low_x,High_x,Shares_y,Low_y,High_y
0,AAPL,80,95,110,50,120,140
1,TSLA,50,80,130,100,100,300


In [263]:
# do the same with pandas function
pd.merge(stocks_2014, stocks_2015, on='Symbol')

,Symbol,Shares_x,Low_x,High_x,Shares_y,Low_y,High_y
0,AAPL,80,95,110,50,120,140
1,TSLA,50,80,130,100,100,300


In [267]:
# give more meaning to the columns by setting a suffic
pd.merge(stocks_2014, stocks_2015, on='Symbol', suffixes=('_2014', '_2015'))

,Symbol,Shares_2014,Low_2014,High_2014,Shares_2015,Low_2015,High_2015
0,AAPL,80,95,110,50,120,140
1,TSLA,50,80,130,100,100,300


In [269]:
# do left join - keep all values in left table
pd.merge(stocks_2014, stocks_2015, how='left', on='Symbol', suffixes=('_2014', '_2015'))

,Symbol,Shares_2014,Low_2014,High_2014,Shares_2015,Low_2015,High_2015
0,AAPL,80,95,110,50.0,120.0,140.0
1,TSLA,50,80,130,100.0,100.0,300.0
2,WMT,40,55,70,NaN,NaN,NaN


In [270]:
# do right join - keep all values in right table
pd.merge(stocks_2014, stocks_2015, how='right', on='Symbol', suffixes=('_2014', '_2015'))

,Symbol,Shares_2014,Low_2014,High_2014,Shares_2015,Low_2015,High_2015
0,AAPL,80.0,95.0,110.0,50,120,140
1,TSLA,50.0,80.0,130.0,100,100,300
2,GE,NaN,NaN,NaN,100,30,40
3,IBM,NaN,NaN,NaN,87,75,95
4,SLB,NaN,NaN,NaN,20,55,85
5,TXN,NaN,NaN,NaN,500,15,23


In [276]:
# outer join - 
stock_outer = pd.merge(stocks_2014, stocks_2015, how='outer', on='Symbol', suffixes=('_2014', '_2015'))

# highlight missing values
stock_outer.style.highlight_null('yellow')

### Use index in `join`
**`join`** is a DataFrame method that works similarly to **`merge`** except that it joins the calling DataFrame's index/column on the **`other`** DataFrame's index. 

In [287]:
stocks_2014.join(stocks_2015.set_index('Symbol'), on='Symbol', lsuffix='_2014', rsuffix='_2015')

,Symbol,Shares_2014,Low_2014,High_2014,Shares_2015,Low_2015,High_2015
0,AAPL,80,95,110,50.0,120.0,140.0
1,TSLA,50,80,130,100.0,100.0,300.0
2,WMT,40,55,70,NaN,NaN,NaN


# End of Section Summary
1. Add a single column
2. Use **`insert`** to add column in specific place
3. Drop a column
4. Automatic alignment happens first with **`pd.concat`**
4. concat by row or column with **`axis`** argument
5. Create multiple levles of indexes to specify them with the **`keys`** argument
6. Change type of alignment with **`outer`** and **`inner`** 
7. Know how to spot a pandas function vs a DataFrame method
8. **`append`** is a dataframe method for simple concatenation. **`pd.concat`** is a function with more power
1. Use **`merge`** to do SQL like joins based on column values. 
1. **`merge`** is both a DataFrame method and a pandas function
1. Use **`join`** to do SQL joins mainly on the index

# Problem Set
We will be working with the city of college admissions dataset for the questions in this notebook. Run the following command before attempting the problems.

In [185]:
import pandas as pd
import numpy as np

college = pd.read_csv('data/college.csv')
pd.options.display.max_columns = 40

### Problem 1
<span  style="color:green; font-size:16px">Insert a column called **`SAT_AVG`** that averages the math and verbal SAT scores before the **`SATVRMID`** column.</span>

In [ ]:
# your code here

### Problem 2
<span  style="color:green; font-size:16px">Read in all three stock csv files and concatenate them horizontally and vertically. Create a hierarchical index that labels each year of data.</span>

In [ ]:
# your code here

### Problem 3
<span  style="color:green; font-size:16px">Take a look at the DataFrame below. Count the total appearances of each letter.</span>

In [434]:
from string import ascii_lowercase
np.random.seed(1)
df = pd.DataFrame(np.random.choice(list(ascii_lowercase), (20,5), replace=True), 
                  columns = ['col1', 'col2', 'col3', 'col4', 'col5'])

df

,col1,col2,col3,col4,col5
0,f,l,m,i,j
1,l,f,p,a,q
2,b,m,h,n,g
3,z,s,u,f,s
4,u,l,k,o,s
5,e,x,x,j,r
6,x,a,w,n,j
7,j,h,w,z,b
8,a,r,i,y,n
9,t,p,k,z,i


In [435]:
# your code here

### Problem 4
<span  style="color:green; font-size:16px">Each Series below represents the amount of TV watched for each sport. Combine all Series so that each column represents a different labeled day. Fill in the missing values with 0. Save it to **`df_sports`**</span>

In [461]:
day1 = pd.Series({'soccer':45, 'basketball':30, 'tennis':10})
day2 = pd.Series({'soccer':55, 'basketball':10, 'bowling':10, 'volleyball':30})
day3 = pd.Series({'soccer':15, 'basketball':20, 'volleyball':40})
day4 = pd.Series({'bowling':100, 'volleyball':20})

In [462]:
# your code here

### Problem 5
<span  style="color:green; font-size:16px">Use **`df_sports`** to find the total TV watched per sport for all the days and also the total amount of TV watched per day. Sort both results from greatest to least.</span>

In [ ]:
# your code here

### Problem 6
<span  style="color:green; font-size:16px">Look up the method **`isnull`** and count the number of nulls per sport.</span>

In [463]:
# your code here

### Problem 7
<span  style="color:green; font-size:16px">Combine all Series again, keeping only the sports that have no missing values for any days.</span>

In [464]:
# your code here